In [1]:
# !pip install htmldate
# !pip install twython
# !pip3 install newspaper3k

In [2]:
import pandas as pd
import numpy as np
import time
import requests
import nltk
import warnings
warnings.filterwarnings('ignore')

from htmldate import find_date
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
from newspaper import Article

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ameyagidh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
!pip install selenium==4.0.0

import selenium

print("Selenium Version:", selenium.__version__)



In [7]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


TypeError: __init__() got an unexpected keyword argument 'chrome_options'

In [4]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# driver = webdriver.Chrome( options=chrome_options)

# # driver.quit()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time  # Don't forget to import time

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)


In [5]:
def get_newslinks(company, page_number):
    """For a given URL, scroll to relevant section to load appropriate HTML into driver,
    iterate through all articles on page and append article URLs to a list

    :param company: name of company to scrape articles for
    :param page_number: page number on news website to iterate over 

    :return: list of articles URLs
    """
    
    url = f"https://www.investing.com/equities/{company}-news/{page_number}"
#     url = f"https://uk.investing.com/equities/{company}-news/{page_number}"
    driver.get(url)

    href = []

    # scroll all the way to the bottom 

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))
        
    cleaned_links = []
    
    # Iterate through all the articles on the page
    for article_number in range(1,11): 
        article = driver.find_element_by_xpath(f'/html/body/div[5]/section/div[8]/article[{article_number}]')
        article_html = article.get_attribute('innerHTML')
        soup = BeautifulSoup(article_html, "lxml")
        for link in soup.find_all('a'): 
            # Get the href
            partial_link = link.get('href')
            if 'https' in partial_link: 
                cleaned_links.append(partial_link)
            # Some links are 'internal' to the page and don't have https in them. The web page must be appended to these links
            elif partial_link[0] == '/': 
                cleaned_links.append('https://uk.investing.com/'+partial_link) 

    return np.unique(cleaned_links)
    

In [6]:
# Create empty list to append URLs 

all_company_urls = []
for page in range(1,119):
    results = get_newslinks('astrazeneca', page)
    all_company_urls.extend(results)
all_company_urls

AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [ ]:
# Save DataFrame 

article_sentiments.to_pickle("datascrapper.pkl")


In [ ]:
article_sentiments.to_csv("azn_article_sentiments_20210105.csv", sep=',', encoding='utf-8', header=True)
# Save URLS to text file

with open('azn_urls_20210105.txt', 'w') as f:
    for link in all_company_urls:
        f.write("%s\n" % link)